In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from pathlib import Path
import zipfile
from shapely import geometry
import os, sys, re
from rasterio.enums import Resampling
from rasterio import mask
from rasterio.io import MemoryFile

In [7]:
def commandline_simulator(dl_sentinel_path, new_save_path, new_raster_name):
    #dl_sentinel_path : downloaded_images/image_name.zip
    #creating the Polygon
    
    cordsUtm = [(437428.46, 5426728.41), (467428.46, 5426728.41), (467428.46, 5396728.41), (437428.46, 5396728.41)]
    poly = geometry.asPolygon(cordsUtm)
    
    #unpacking the `.zip` archive
    
    zipName = dl_sentinel_path
    with zipfile.ZipFile(zipName,"r") as fz:
        fz.extractall('downloaded_images/')
        
    
    folder = Path( dl_sentinel_path[:-3]+'SAFE/' + 'GRANULE/L1C_T31UDQ_A022099_20210530T104621/IMG_DATA/')
    
    #creating the dataframe corresponding to our images with different resolutions
    bands = []
    for f in folder.glob('*/*B*'):
        tmp = list(re.search(r'(B.{2})_{1}(\d\d)',f.name).groups())
        tmp.append(f.as_posix())
        bands.append(tmp)
    
    df = pd.DataFrame(bands,columns=['name','resolu','add'])
    df = df.groupby('name').min()
    
    #defining the outFold
    outFold = Path(new_save_path)
    if not outFold.is_dir():
        outFold.mkdir()
    
    #creating the metaData
    
    metaOut = None
    metaTmp = None

    with rasterio.open(df.iloc[1,-1]) as src: 
        metaOut = src.meta
        metaOut['driver'] = 'GTiff'
        metaOut['nodata'] = 2**12
        metaOut['count'] = df.shape[0]

        metaTmp = src.meta
        metaTmp['count'] = 1

        outShape = (metaOut['height'],metaOut['width'])

    
    #Resampling and clipping
    
    outData = []

    for i in range(df.shape[0]):
        add = df.iloc[i,1]

        with rasterio.open(add) as src:                 
            if int(df.iloc[i,0]) > 10:
                data = src.read(1,out_shape = outShape, resampling = Resampling.nearest)
                # we resample data with resolution >10
            else:
                data = src.read(1)
        #print(data.shape)


        with MemoryFile() as memfile:
            with memfile.open(**metaTmp) as dataset:
                dataset.write(data,1)
                out_image, out_transform = mask.mask(dataset, [poly], crop=True) #clipping area of interest
                out_image = np.transpose(out_image,(1,2,0))
                out_image = out_image[:,:,0].copy()
                outData.append(out_image)

    metaOut.update({"height": out_image.shape[0],
                     "width": out_image.shape[1],
                     "transform": out_transform})
    
    #saving the clipped channels as a new multi-channel raster in `GeoTIFF` format
    
    dst_addres = new_save_path+new_raster_name

    with rasterio.open(dst_addres,'w',**metaOut) as dst:
        for i,ar in enumerate(outData,1):
            dst.write(ar,i)
            dst.set_band_description(i,df.index[i-1])


In [8]:
commandline_simulator('downloaded_images/S2B_MSIL1C_20210530T104619_N0300_R051_T31UDQ_20210530T115735.zip','downloaded_images/','Ready_raster2.tif')

IndexError: single positional indexer is out-of-bounds